In [ ]:
import numpy as np
import pandas as pd

# Load RI data

In [ ]:
riDF= pd.read_csv('./ri-subscriptions.csv')

In [ ]:
riDF.head()

In [ ]:
len(riDF['Instance Type'].unique())

In [ ]:
riDF = riDF[['Instance Type','RI Utilization','Number of RIs']]

In [ ]:
riDF

In [ ]:
riDF[riDF['Instance Type']=='m4.large']

In [ ]:
# ri.describe()

# Load Instance data

In [ ]:
instanceDF = pd.read_csv('./dataframes/instanceData.csv')

In [ ]:
# Lets see how many r3.xlarge I have since that type isn't in the RI data
(instanceDF[instanceDF.type=='r3.xlarge'])

# Load all ec2 data

In [ ]:
ec2DF = pd.read_csv('./all_ec2_raquel.csv',header=None)

In [ ]:
ec2DF.columns = ['id','state','type']

In [ ]:
len(ec2DF.id.unique())

In [ ]:
ec2s=ec2DF.groupby(['type']).count()
ec2s = ec2s[['id']]
ec2s.columns=['Count']
ec2s

## Obtain avg RI utilization and RI count per type

In [ ]:
avgUtiliz=riDF.groupby('Instance Type').mean()
avgUtiliz=avgUtiliz.drop(['Number of RIs'],axis=1)
avgUtiliz

In [ ]:
totalRIs=riDF.groupby('Instance Type').sum()
totalRIs=totalRIs.drop(['RI Utilization'],axis=1)
totalRIs

## Make a dataframe for prices

In [ ]:
aws = pd.read_csv('./awsPrices.csv')

In [ ]:
aws.head()

In [ ]:
merge1 = pd.merge(aws,avgUtiliz,how='left',left_on=['type'], right_on=['Instance Type'])       #Add avgUtiliz
merge2 = pd.merge(merge1, totalRIs, how='left', left_on=['type'], right_on=['Instance Type'])  #Add totalRIs 
merge3 = pd.merge(merge2, ec2s, how='inner', on=['type'])                                      #Add ec2 Count
merge3

In [ ]:
costDF = merge3[['type','Price3','RI Utilization','Number of RIs','Count']]
costDF.columns=['type','upfront3', 'utilizRI','countRI','countEC2']
costDF.to_csv('./dataframes/costData.csv', index=False)

In [ ]:
costDF

In [ ]:
costDF['countRI']/costDF['countEC2']

In [ ]:
# Make formula
# if RI utiliz <1 (100%) then the cost of the instance of that type is only going to be corresponding part of RI cost
# if RI utiliz>=1 then cost will be ProbRI*RIcostProportion + ProbOnDemand*ODcost

def calculateCost(i_type):
    if avgUtiliz.log[i_type]['RI Utilization']<1 :
        price = awsPrices.loc[i_type]

